# Dashboard Tiles

In [ ]:
test_id = 28971

## A. Credit allocation

In [ ]:
def solvability_score(train_df, test_df, model, test_id):
    '''
    Return the probability in % for the sample to be attributed the note 1.
    model must be already fitted to the train set.
    '''
    temp_df = test_df.copy()
    temp_df['Prediction_probability'] = model.predict_proba(test_df)[:,1]
    app_predict_proba = temp_df.loc[test_id]['Prediction_probability']
    app_predict_proba = round(app_predict_proba, 2) * 100
    return app_predict_proba


def credit_allocation(train_df, test_df, model, test_id):
    '''

    '''
    score = solvability_score(train_samp_df, test_samp_df, rdm_best, test_id)
    if score < 45 and score >= 0:
        note = 'red'
    elif score < 50:
        note = 'orange'
    elif score <= 100:
        note='green'
    else:
        return print('Error in score evaluation')
    return note

In [ ]:
credit_allocation(train_samp_df, test_samp_df, rdm_best, test_id)

## B. Solvability

In [ ]:
solvability_score(train_samp_df, test_samp_df, rdm_best, test_id)

## C. Recommandations

## D. Settings

In [ ]:
# List of features that the applicant can potentially change


In [ ]:
# External data that can be used


## E. Strengths & weaknesses

In [ ]:
def plot_customer_position(train_df, test_df, model, test_id):
    '''
    Plot the position of the applicant among the population of former
    applicants.
    '''
    # TEMPORARY: select a feature
    features = get_sample_features(test_df, model, test_id)
    feat = features[11]
    # Determine if the feature is quantitative or qualitative
    if train_df[feat].nunique()==2:
        return cust_pos_qual_feat(train_df, test_df, model, feat, test_id)
    else:
        return cust_pos_quant_feat(train_df, test_df, model, feat, test_id)

def get_sample_features(test_df, model, test_id):
    '''
    Given a dataframe and a classification model, returns the list of the most
    important features.
    The model must be already trained on a train set.
    '''
    features = most_important_features(model, X, 20)
    features = list(features.index)
    return features


def get_sample_judgement(test_df, model, test_id):
    '''
    Determine if the applicant is eligible or not.
    '''
    temp_df = test_df.copy()
    temp_df['Judgement'] = model.predict(test_df)
    applicant_judgement = temp_df.loc[test_id]['Judgement']
    applicant_judgement = int(applicant_judgement)
    return applicant_judgement


def cust_pos_quant_feat(train_df, test_df, model, feature, test_id):
    '''
    Plot the position of the choosen customer among the population of other
    customers, for a quantitative feature.
    '''
    # POSITION OF THE APPLICANT
    applicant_row = test_df.loc[test_id]
    applicant_feature_value = applicant_row[feature]
    applicant_feature_value = round(applicant_feature_value, 3)
    # Distribution de la caractéristique dans le TRAIN set pour:
    # - les targets à 0
    # - les targets à 1
    train_0_feature = list(train_df[train_df['TARGET']==0][feature])
    train_1_feature = list(train_df[train_df['TARGET']==1][feature])
    # 
    n_0, bins_0, patches_0 = plt.hist(train_0_feature,
                                      bins=30,
                                      edgecolor='k',
                                      color='gray',
                                      alpha=0.6)
    n_1, bins_1, patches_1 = plt.hist(train_1_feature,
                                      bins=30,
                                      edgecolor='k',
                                      color='green',
                                      alpha=0.6)
    # Special coloration for applicant identification
    judgement = get_sample_judgement(test_df, model, test_id)
    if judgement == 1: # applicant is accepted
        plt.title(feature + ', ' + str(applicant_feature_value) + ' - Granted')
        for i, element in enumerate(patches_1):
            if element.xy[0] > applicant_feature_value:
                n_print = i
                break
        patches_1[i].set_fc('b')
    elif judgement == 0: # applicant is refused
        plt.title(feature + ', ' + str(applicant_feature_value) + ' - Refused')
        for i, element in enumerate(patches_0):
            if element.xy[0] > applicant_feature_value:
                n_print = i
                break
        patches_0[i].set_fc('b')
    # Other plotting parameters
    plt.legend(['Refused',
                'Granted'])
    plt.show()


def cust_pos_qual_feat(train_df, test_df, model, feature, test_id):
    '''
    Plot the position of the choosen customer among the population of other
    customers, for a quantitative feature.
    
    Warning: it is not very accurate to use app_train_df, as it corresponds
    to the very original dataframe, without the feature engineering
    (nan values, outliers, ...).
    '''
    # Get the corresponding value for the customer
    value = test_df.loc[test_id][feature]
    # Get the root feature
    for col in cat_col['Categorical feature name']:
        if col in feature:
            original_col = col
    # Get the value taken by the candidate
    if value == 0:
        # Get the list of columns coming from the same original_col
        similar_feats = []
        for col in test_df.columns:
            if original_col in col:
                similar_feats.append(col)
        # Among these columns, find the one that contains 1
        for col in similar_feats:
            if test_df[col].loc[test_id] == 1:
                app_category = col
                app_category = app_category.split(original_col)[-1]
    else:
        app_category = feature.split(original_col)[-1]
    if app_category:
        app_category = app_category[1:].replace('_', ' ')
    # Get the categorical distribution for accepted applicants
    train_1_df = app_train_df[app_train_df['TARGET']==1]
    train_1_df = pd.DataFrame(train_1_df[original_col].value_counts())
    # Get the categorical distribution for refused applicants
    train_0_df = app_train_df[app_train_df['TARGET']==0]
    train_0_df = pd.DataFrame(train_0_df[original_col].value_counts())
    # Merge the two dataframe
    pareto_df = pd.merge(train_1_df,
                         train_0_df,
                         how='outer',
                         left_index=True,
                         right_index=True,
                         suffixes=['_accepted', '_refused'])
    pareto_df.replace(np.nan, 0, inplace=True)
    # Create a column sum
    pareto_df['Ratio'] = pareto_df[pareto_df.columns[1]] / pareto_df[pareto_df.columns[0]]
    pareto_df.sort_values(by='Ratio', ascending=False, inplace=True)   
    # Plot parameters
    plt.title(original_col + ' - Chance of success')
    plt.xlabel('Categories')
    plt.xticks(rotation=45,
               ha='right')
    plt.ylabel('Percentage of granted credits')
    # Color the semi-column of the customer choosen.
    colors=[]
    for category in pareto_df.index:
        if category == app_category:
            colors.append('blue')
        else:
            colors.append('gray')
    # Plot the bar graph
    plt.bar(pareto_df.index,
            pareto_df['Ratio'],
            alpha=0.5,
            color=colors,
            edgecolor='k')
    # Other parameters
    plt.show()

In [ ]:
plot_customer_position(train_df, test_samp_df, rdm_best, test_id)

In [ ]:
cust_pos_qual_feat(train_df, test_samp_df, rdm_best, 'NAME_TYPE_SUITE_Family', test_id)